In [1]:
import sys
import numpy as np
import torch as pt
import matplotlib.pyplot as plt
import sys
from pathlib import Path
sys.path.append(Path('../deterministic'))
from deterministic.mps import MPS
import src.constants as constants
from display_data.data_acquisition_shadow import derandomized_classical_shadow, randomized_classical_shadow
from display_data.prediction_shadow import estimate_exp

In [2]:
qubit_num = 10
psi = pt.rand(2**qubit_num, dtype=pt.cdouble)
mps = MPS.from_state_vector(qubit_num, psi)
print(mps)

MPS None:
	visible_num = 10
	phys_dims = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
	bond_dims = [2, 4, 8, 16, 32, 16, 8, 4, 2]
	ext_bond_dims = [1, 2, 4, 8, 16, 32, 16, 8, 4, 2, 1]
	orth_idx = None



In [2]:
#generate a random tensor list
qubit_num = 12
bond_dim = 50
tensor_liste = [pt.rand([1, 2, bond_dim], dtype=pt.cdouble)]
for idx in range(qubit_num - 2):
    tensor_liste.append(pt.rand([bond_dim, 2, bond_dim], dtype=pt.cdouble))
tensor_liste.append(pt.rand([bond_dim, 2, 1], dtype=pt.cdouble))

In [12]:
#compute the norm of the tensor network
def norm(qubit_num, tensor_list):
    result = pt.einsum('abc,abj->cj', tensor_list[0], tensor_list[0].conj())
    for idx in range(1, qubit_num-1):
        result = pt.einsum('cj,cab->jab', result, tensor_list[idx])
        result = pt.einsum('jab,jad->bd', result, tensor_list[idx].conj())
    result = pt.einsum('ab,acj->bcj', result, tensor_list[qubit_num - 1])
    result = pt.einsum('bcj,bci->ji', result, tensor_list[qubit_num - 1].conj())
    return pt.sqrt(result)

print(norm(qubit_num=10, tensor_list=tensor_liste))

tensor([[5.5081e+11+1.5233e-05j]], dtype=torch.complex128)


In [20]:
mps = MPS.from_tensor_list(tensor_liste)
print(mps.norm())
# -> the result is the same as it should be

tensor(1.8514e+15+0.0196j, dtype=torch.complex128)


In [15]:
print(norm(10, mps.tensors))

tensor([[25.9219+0.j]], dtype=torch.complex128)


In [9]:
#make tensor specified by idx center of orthogonality
def canonicalise(tensor_list, idx, D):
    #from the left
    for index in range(0, idx):
        bond_dim_left = tensor_list[index][:,0,0].size()[0]
        bond_dim_right = tensor_list[index][0,0,:].size()[0]
        Qm, R = pt.linalg.qr(tensor_list[index].reshape(bond_dim_left*D, bond_dim_right))
        tensor_list[index] = pt.reshape(Qm, (bond_dim_left, D, Qm.size()[1]))
        tensor_list[index + 1] = pt.einsum('ab,bcd->acd', R, tensor_list[index + 1])
    for index in range(idx, len(tensor_list) - 1):
        index = len(tensor_list) - 1 - index + idx #because we want to start from the right
        bond_dim_left = tensor_list[index][:,0,0].size()[0]
        bond_dim_right = tensor_list[index][0,0,:].size()[0]
        Qm_t, R_t = pt.linalg.qr(pt.t(tensor_list[index].reshape(bond_dim_left, bond_dim_right * D)))
        Qm = pt.t(Qm_t)
        R = pt.t(R_t)
        tensor_list[index] = pt.reshape(Qm, (Qm_t.size()[1], D, bond_dim_right))
        tensor_list[index - 1] = pt.einsum('abc,cd->abd', tensor_list[index - 1], R)
    return tensor_list

print(canonicalise(tensor_list=tensor_liste, idx=9, D=2)[9])


tensor([[[-2.0688e+11-3.3123e+11j],
         [-2.0574e+11-3.2946e+11j]],

        [[-2.9711e+08+2.2473e+08j],
         [ 7.4502e+08+4.2499e+07j]]], dtype=torch.complex128)


In [10]:
print(pt.linalg.norm(canonicalise(tensor_list=MPS.from_tensor_list(tensor_liste).tensors, idx=4, D=2)[4]))
print(norm(10, tensor_liste))
# -> the results agree as they should

tensor(5.0636e+11, dtype=torch.float64)
tensor([[5.0636e+11-6.6266e-05j]], dtype=torch.complex128)


In [3]:
def canonicalise_left_to_index(tensor_list, idx, phys_dim):
    #from the left
    for index in range(0, idx):
        bond_dim_left = tensor_list[index][:,0,0].size()[0]
        bond_dim_right = tensor_list[index][0,0,:].size()[0]
        Qm, R = pt.linalg.qr(tensor_list[index].reshape(bond_dim_left*phys_dim, bond_dim_right))
        tensor_list[index] = pt.reshape(Qm, (bond_dim_left, phys_dim, Qm.size()[1]))
        tensor_list[index + 1] = pt.einsum('ab,bcd->acd', R, tensor_list[index + 1])
    return tensor_list

In [204]:
# sampling algorithm
def sampling(tensor_list, qubit_num, num_samples):
    bits_sampled = pt.zeros((qubit_num, num_samples))
    probabilities_for_bits = pt.ones((qubit_num, num_samples))
    canonicalised_tensors = canonicalise_left_to_index(tensor_list, qubit_num-1, 2)
    #we only need to do this step if the MPS is not normalised
    part_func = pt.einsum('ijk,ijl->kl', canonicalised_tensors[qubit_num-1], canonicalised_tensors[qubit_num-1].conj())[0,0]

    for index in range(qubit_num):
        idx = qubit_num - 1 -index
        # because the probabilities for all samples is different, we cannot draw them all at once, but have to draw them one by one by looping
        for k in range(num_samples):
        #contract the network
            if idx==qubit_num-1:
                result = pt.einsum('ijl,iml->jm', canonicalised_tensors[idx], canonicalised_tensors[idx].conj())
            else:
                result = pt.einsum('fh,jh->fj', canonicalised_tensors[qubit_num-1][:,int(bits_sampled[qubit_num-1, k].item()),:], canonicalised_tensors[qubit_num-1][:,int(bits_sampled[qubit_num-1, k].item()),:].conj())
                for counter in range(qubit_num - 1 - idx - 1):
                    index = qubit_num - 1 - counter - 1
                    result = pt.einsum('fj,df->dj', result, canonicalised_tensors[index][:,int(bits_sampled[index, k].item()),:])
                    result = pt.einsum('dj,lj->dl', result, canonicalised_tensors[index][:,int(bits_sampled[index, k].item()),:].conj())
                result = pt.einsum('rs,acr->acs', result, canonicalised_tensors[idx])
                result = pt.einsum('acs,ams->cm', result, canonicalised_tensors[idx].conj())
        #contraction done
            prob_for_previous_bits = pt.prod(probabilities_for_bits[:,k])
            probs = [result[0,0]/part_func/prob_for_previous_bits, result[1,1]/part_func/prob_for_previous_bits]
            bits_sampled[idx, k] = pt.multinomial(pt.tensor([probs[0].real.item(), probs[1].real.item()]), 1, replacement=True)[0].item()
            probabilities_for_bits[idx, k] = probs[int(bits_sampled[idx, k].item())]
    return bits_sampled, probabilities_for_bits

In [206]:
%timeit sampling(tensor_liste, qubit_num, 100)


1.76 s ± 44.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [178]:
# sampling algorithm
def sampling_one_sample(tensor_list, qubit_num):
    bits_sampled = pt.zeros(qubit_num)
    probabilities_for_bits = pt.ones(qubit_num)
    #we only need to do this step if the MPS is not normalised
    canonicalised_tensors = canonicalise_left_to_index(tensor_list, qubit_num-1, 2)
    part_func = pt.einsum('ijk,ijl->kl', canonicalised_tensors[qubit_num-1], canonicalised_tensors[qubit_num-1].conj())[0,0]

    for index in range(qubit_num):
        idx = qubit_num - 1 -index
        canonicalised_tensors = canonicalise_left_to_index(tensor_list, idx, 2)
        #contract the network
        if idx==qubit_num-1:
            result = pt.einsum('ijl,iml->jm', canonicalised_tensors[idx], canonicalised_tensors[idx].conj())
        else:
            result = pt.einsum('fh,jh->fj', canonicalised_tensors[qubit_num-1][:,int(bits_sampled[qubit_num-1].item()),:], canonicalised_tensors[qubit_num-1][:,int(bits_sampled[qubit_num-1].item()),:].conj())
            for counter in range(qubit_num - 1 - idx - 1):
                index = qubit_num - 1 - counter - 1
                result = pt.einsum('fj,df->dj', result, canonicalised_tensors[index][:,int(bits_sampled[index].item()),:])
                result = pt.einsum('dj,lj->dl', result, canonicalised_tensors[index][:,int(bits_sampled[index].item()),:].conj())
            result = pt.einsum('rs,acr->acs', result, canonicalised_tensors[idx])
            result = pt.einsum('acs,ams->cm', result, canonicalised_tensors[idx].conj())
        #contraction done
        prob_for_previous_bits = pt.prod(probabilities_for_bits, dim=0)
        probs = [result[0,0]/part_func/prob_for_previous_bits, result[1,1]/part_func/prob_for_previous_bits]
        # because the probabilities for all samples is different, we cannot draw them all at once, but have to
        bits_sampled[idx] = pt.multinomial(pt.tensor([probs[0].real.item(), probs[1].real.item()]), 1, replacement=True)[0].item()
        probabilities_for_bits[idx] = probs[int(bits_sampled[idx].item())]
    return bits_sampled, probabilities_for_bits

In [202]:
%timeit sampling_one_sample(tensor_liste, qubit_num)

60 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [207]:
# apply a rotation
def rotation(tensoren, rotation_pauli_string):
    rot_tensors = []
    for idx in range(len(tensoren)):
        rot_tensors.append(pt.einsum('ab,cbd->cad',constants.PAULI_ROT[rotation_pauli_string[idx]], tensoren[idx]))
    return rot_tensors

In [218]:
# shadow generation
def generate_shadow_and_prob(tensoren, qubit_num, number_of_measurements, num_measurements_per_rot):
    measurement_outcomes = []
    probabilities = []
    measurement_procedure = randomized_classical_shadow(number_of_measurements, qubit_num)
    for i in range(number_of_measurements):
        tensor_list_rot = rotation(tensoren, measurement_procedure[i])
        measurement_bits, probs = sampling(tensor_list_rot, qubit_num, num_measurements_per_rot)
        probabilities.append(pt.prod(probs, dim = 0))
        #convert binary torch tensor to index
        measurement_outcome = pt.zeros(num_measurements_per_rot, dtype = pt.int)
        for k in range(0, qubit_num):
            measurement_outcome[:] = measurement_outcome + measurement_bits[k] * 2**(qubit_num - 1 - k)
        measurement_outcomes.append(measurement_outcome)
    return measurement_outcomes, measurement_procedure, probabilities

In [223]:
print(generate_shadow_and_prob(tensor_liste, qubit_num, 2, 10))

([tensor([ 1365, 15703,  6816, 10152, 11413,  1237,  7263,  9687,  8042,  5533],
       dtype=torch.int32), tensor([ 8360, 15422,   835,  9894,  5835,  2507,  2001,  4040, 15787,  5575],
       dtype=torch.int32)], [['X', 'Y', 'Y', 'Y', 'Z', 'X', 'Y', 'X', 'Z', 'Z', 'Y', 'Z', 'X', 'Z'], ['Z', 'X', 'Y', 'X', 'Y', 'Y', 'X', 'Z', 'Z', 'Y', 'X', 'Y', 'X', 'Y']], [tensor([0.0011, 0.0016, 0.0006, 0.0008, 0.0011, 0.0010, 0.0011, 0.0015, 0.0008,
        0.0010]), tensor([0.0002, 0.0002, 0.0003, 0.0002, 0.0004, 0.0003, 0.0003, 0.0003, 0.0002,
        0.0004])])


In [94]:
# this function converts the stored measurement outcomes to the correct shape needed for the shadow prediction
def conversion_to_prediction_shadow_dict_shape(measurement_procedure, measurement_index, qubit_num):
    to_str_func = np.vectorize(lambda x: np.binary_repr(x).zfill(qubit_num))
    strs = to_str_func(measurement_index)
    dirac_rep = np.zeros(list(measurement_index.shape) + [qubit_num], dtype=np.int8)
    for bit_ix in range(0, qubit_num):
        fetch_bit_func = np.vectorize(lambda x: x[bit_ix] == '1')
        dirac_rep[...,bit_ix] = fetch_bit_func(strs).astype("int8")
    measurement_array = np.where(dirac_rep == 1, -1, dirac_rep)
    measurement_array = np.where(dirac_rep == 0, 1, measurement_array)
    measurement = np.dstack((measurement_procedure, np.array(measurement_array, dtype=int)))
    return measurement

In [158]:
#now we estimate the energy from the classical shadows, we pick the open boundary TFIM Hamiltonian
def ising_energy_shadows(measurements, probabilities, qubit_num, ratio_h_J):
    h_val = ratio_h_J
    j_val = 1
    observables = []
    for i in range(0, qubit_num):
        x_arr = [['X', i]]
        observables.append(x_arr)
        if i <= qubit_num - 2:
            z_arr = [['Z', i], ['Z', i + 1]]
            observables.append(z_arr)


    energies = np.zeros(len(measurements))
    for j in range(len(measurements)):
        for i in range(0, len(observables)):
            sum_product, cnt_match = estimate_exp([measurements[j]], observables[i])
            if sum_product == 0 and cnt_match == 0:
                expectation_val = 0
            elif cnt_match == 0 and sum_product != 0:
                print('cnt_match is zero (problemo)!')
            else:
                expectation_val =  sum_product / cnt_match * probabilities[j]
            if i % 2 == 0:
                energies[j] = energies[j] + h_val * expectation_val * 3 # weighing with factor 3**locality to counteract the chance of hitting
            else:
                energies[j] = energies[j] + j_val * expectation_val * 3**2
        energies[j] = energies[j] / probabilities[j]
    energy = np.mean(energies)
    return energy

In [131]:
def energy_shadow(measurement, ratio_h_j, qubit_num):
    observables = []
    for i in range(0, qubit_num):
        x_arr = [['X', i]]
        observables.append(x_arr)
        if i <= qubit_num - 2:
            z_arr = [['Z', i], ['Z', i + 1]]
            observables.append(z_arr)

    # now we have our measurement outcome and our observables stored in the correct format
    energy = 0
    for i in range(0, len(observables)):
        sum_product, cnt_match = estimate_exp(measurement, observables[i])
        if sum_product == 0 and cnt_match == 0:
            expectation_val = 0
        elif cnt_match == 0 and sum_product != 0:
            print('cnt_match is zero (problemo)!')
        else:
            expectation_val = sum_product / cnt_match
        if i % 2 == 0:
            energy = energy + ratio_h_j * expectation_val
        else:
            energy = energy + expectation_val
    return energy

In [176]:
ratio_h_j = 0.1
qubit_num = 14
save_direc_groundstate = Path('data/Diagonal_hamiltonian/groundstate_dict.npy')
ground_state_dict = np.load(save_direc_groundstate, allow_pickle='TRUE')
ground_state_dict = ground_state_dict.item()
ground_state = ground_state_dict[str(1)]['open'][str(qubit_num)][str(ratio_h_j)]
mps = MPS.from_state_vector(qubit_num, ground_state)
tensor_liste = mps.tensors

In [160]:
meas_outcomes, meas_procedure, probs = generate_shadow_and_prob(tensor_liste, qubit_num, 500)
measurements = conversion_to_prediction_shadow_dict_shape(meas_procedure, np.array(meas_outcomes), qubit_num)

In [161]:
print(ising_energy_shadows(measurements, probs, qubit_num , ratio_h_j))
print(energy_shadow(measurements, ratio_h_j, qubit_num))

-12.4338
-13.064391293960684


In [56]:
save_direc_eigs = Path('data/Diagonal_hamiltonian/eigenvalues_first_three_dict.npy')
energy_dict = np.load(save_direc_eigs, allow_pickle=True)
energy_dict = energy_dict.item()
energy_brute = energy_dict[str(1)]['open'][str(qubit_num)][str(ratio_h_j)][0]
print(energy_brute)

-13.040028203462525


In [43]:
print(measurements)

[[['Y' '-1']
  ['Z' '1']
  ['Y' '-1']
  ...
  ['Z' '-1']
  ['Z' '1']
  ['Z' '-1']]

 [['X' '-1']
  ['Z' '1']
  ['X' '1']
  ...
  ['Z' '1']
  ['X' '-1']
  ['Y' '1']]

 [['Z' '-1']
  ['Z' '1']
  ['Y' '1']
  ...
  ['X' '-1']
  ['Z' '-1']
  ['X' '-1']]

 ...

 [['Z' '-1']
  ['X' '1']
  ['Y' '1']
  ...
  ['X' '-1']
  ['X' '-1']
  ['X' '1']]

 [['Y' '1']
  ['Z' '1']
  ['X' '-1']
  ...
  ['Y' '-1']
  ['X' '1']
  ['X' '-1']]

 [['Y' '1']
  ['X' '-1']
  ['Y' '-1']
  ...
  ['Z' '1']
  ['Y' '-1']
  ['Z' '-1']]]


In [74]:
observables = []
for i in range(0, qubit_num):
    x_arr = [['X', i]]
    observables.append(x_arr)
    if i <= qubit_num - 2:
        z_arr = [['Z', i], ['Z', i + 1]]
        observables.append(z_arr)
    else:
        z_arr = None
print(observables)

[[['X', 0]], [['Z', 0], ['Z', 1]], [['X', 1]], [['Z', 1], ['Z', 2]], [['X', 2]], [['Z', 2], ['Z', 3]], [['X', 3]], [['Z', 3], ['Z', 4]], [['X', 4]], [['Z', 4], ['Z', 5]], [['X', 5]], [['Z', 5], ['Z', 6]], [['X', 6]], [['Z', 6], ['Z', 7]], [['X', 7]], [['Z', 7], ['Z', 8]], [['X', 8]], [['Z', 8], ['Z', 9]], [['X', 9]], [['Z', 9], ['Z', 10]], [['X', 10]], [['Z', 10], ['Z', 11]], [['X', 11]], [['Z', 11], ['Z', 12]], [['X', 12]], [['Z', 12], ['Z', 13]], [['X', 13]]]
